#**GEFS Eclipse**
<a href="https://githubtocolab.com/csteele2/Wx4Colab/blob/master/GEFS_Eclipse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a> <br/>
A notebook to download, process, and plot probabilities of sky cover from the Global Ensemble Forecast System (GEFS - aka "American Ensemble") for the April 2024 solar eclipse. Uses some code borrowed from [Brian Blaylock](https://github.com/blaylockbk) to more efficiently access partial grib files from [GEFS grib archive on AWS](https://aws.amazon.com/marketplace/pp/prodview-qumzmkzc2acri#resources) for data, and matplotlib, cartopy, and contextily to plot. This was quickly hacked together from other code, so I make no guarantees. Note the quarter-degree files on AWS only go out to 240 hours, so this will only plot that far out. Be sure to click the folder icon on the bar to the left to get your PNGs!<br/>
<br/>
Caleb Steele - https://github.com/csteele2/Wx4Colab
<br/>

## **1 - Install and Import**

This will install and import everything we need. You need only run this once per session, then you can make all the changes to the form and make as many plots as you wish without having to rerun this cell (unless your session expires and spins down).

In [ ]:
!pip install ecmwflibs
!pip install eccodes
!pip install cfgrib
!pip install cartopy contextily pyproj pyepsg zarr xarray tqdm

from datetime import datetime, timedelta  # our data is time-based
import xarray as xr  # easily create hypercubes and store/read from zarrs
import numpy as np  # for python stuff that deals with numbers
import os, sys, getopt, re  # to manage the temporary files and read in arguments
import glob  # to help clean up
from urllib.request import urlretrieve # to get files

import warnings  # to squash warnings that I have deemed insignificant
import requests # just to sniff for a remote file first
import time # in case we need to wait for GEFS data

from cartopy import crs as ccrs, feature as cfeature
import cartopy.geodesic as geodesic
import cartopy.io.shapereader as shpreader
from cartopy.feature import ShapelyFeature
import contextily as cx
from pyproj.crs import CRS # to warp maptiles
import matplotlib  # to make plots, duh
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects  # to add outline to text, etc.
import matplotlib.patches as mpatches  # to make a nicer looking legend vice a colorbar
from tqdm.notebook import tqdm # to show progress and minimize printed output

warnings.filterwarnings("ignore")

## **2 - Edit Form Options & Go!**

The cell below has some config things to set. For the curious, you can unhide the code hiding underneath.

In [ ]:
# @title Chose Options
#@markdown Select a GEFS Cycle
init_date = "2024-03-29" #@param {type:"date"}
init_hour = 18 #@param {type:"slider", min:0, max:18, step:6}
model_run = datetime.strptime(init_date,'%Y-%m-%d') + timedelta(hours=int(init_hour))
#@markdown <br />

#@markdown Pick a sky threshold (the NDFD threshold for sunny/clear is 12.5)
sky_threshold = 12 # @param {type:"integer"}
#@markdown <br />

#@markdown Pick a map theme
map_theme = "Dark Grey Matter" #@param ["Light Shaded Relief", "Positron", "Dark Matter", "Dark Grey Matter", "ESRI Light Grey", "ESRI Dark Grey"]
if "Dark" in map_theme:
  font_color='w'
  face_color='#272727'
else:
  font_color = 'k'
  face_color = 'w'
#@markdown Set the map scale offset from default (i.e. a 1 would scale up one level [make labels bigger])
map_scale_offset = "0" #@param ["-2","-1", "0", "1","2"]
map_zoom_offset = int(map_scale_offset)
dom = "ECONUS" #@param ["CONUS", "ECONUS", "---- WESTERN REGION ----", "WR","NR","UT","AZ","SWUS","PNW", "---- CENTRAL REGION ----","CR", "NP", "GL", "CUS", "CO", "---- SOUTHERN REGION ----", "SR", "TXOK", "SE", "---- EASTERN REGION ----", "ER", "NE"]


members = ["c00"] + [f"p{i:02}" for i in range(1, 31)]
members_tqdm = tqdm(["c00"] + [f"p{i:02}" for i in range(1, 31)])
eclipse_valid_times = [datetime(2024, 4, 8, 15, 0), datetime(2024, 4, 8, 18, 0), datetime(2024, 4, 8, 21, 0), datetime(2024, 4, 9, 0, 0)]
fh_tds = [t - model_run for t in eclipse_valid_times]
fh_list = [int(d.total_seconds() / 3600) for d in fh_tds]

sfc_variables = ["tcdc"]

if not os.path.exists("data"):
    os.makedirs("data")
if not os.path.exists("tmpcache"):
    os.makedirs("tmpcache")
if not os.path.exists("shp"):
    os.makedirs("shp")

for file in ["center", "upath_hi"]:
  for extension in [".shp", ".dbf", ".prj", ".shx"]:
    if os.path.exists(f"shp/{file}{extension}"):
      pass
    else:
        urlretrieve(f"https://www.dynamicmeteorology.com/data/2024eclipse_shapefiles/{file}{extension}", f"shp/{file}{extension}")

def download_subset(parameter, level, member, forecast_hour, local_filename):
    # print("   > Downloading a subset of GEFS")
    gribset = "pgrb2sp25"
    gribfilekey = "pgrb2s.0p25"
    prefix = f"gefs.{model_run.strftime('%Y%m%d/%H')}/atmos/{gribset}/"
    local_file = os.path.join("tmpcache", local_filename)
    aws_root = f"https://noaa-gefs-pds.s3.amazonaws.com/{prefix}"
    remote_url = f"{aws_root}ge{member}.t{model_run.strftime('%Hz')}.{gribfilekey}.f{int(forecast_hour):03}"
    # print(remote_url)
    searchStringDict = {
        "tcdc": ":TCDC:",
    }
    # print(searchStringDict[parameter])
    search_string = searchStringDict[parameter]
    # print(search_string)
    idx = remote_url + ".idx"
    r = requests.get(idx)
    if not r.ok:
        print("     ❌ SORRY! Status Code:", r.status_code, r.reason)
        print(f"      ❌ It does not look like the index file exists: {idx}")

    lines = r.text.split("\n")
    expr = re.compile(search_string)
    byte_ranges = {}
    for n, line in enumerate(lines, start=1):
        # n is the line number (starting from 1) so that when we call for
        # `lines[n]` it will give us the next line. (Clear as mud??)

        # Use the compiled regular expression to search the line
        if expr.search(line):
            # aka, if the line contains the string we are looking for...

            # Get the beginning byte in the line we found
            parts = line.split(":")
            rangestart = int(parts[1])

            # Get the beginning byte in the next line...
            if n + 1 < len(lines):
                # ...if there is a next line
                parts = lines[n].split(":")
                rangeend = int(parts[1])
            else:
                # ...if there isn't a next line, then go to the end of the file.
                rangeend = ""

            # Store the byte-range string in our dictionary,
            # and keep the line information too so we can refer back to it.
            byte_ranges[f"{rangestart}-{rangeend}"] = line
            # print(line)
    for i, (byteRange, line) in enumerate(byte_ranges.items()):

        if i == 0:
            # If we are working on the first item, overwrite the existing file.
            curl = f"curl -s --range {byteRange} {remote_url} > {local_file}"
        else:
            # If we are working on not the first item, append the existing file.
            curl = f"curl -s --range {byteRange} {remote_url} >> {local_file}"
        try:
            num, byte, date, var, level, forecast, _ = line.split(":")
        except:
            pass

        # print(f'  Downloading GRIB line [{num:>3}]: variable={var}, level={level}, forecast={forecast}')
        os.system(curl)

    if os.path.exists(local_file):
        # print(f'      ✅ Success! Searched for [{searchStringDict[parameter]}] and got [{len(byte_ranges)}] GRIB fields and saved as {local_file}')
        return local_file
    else:
        print(
            print(
                f"      ❌ Unsuccessful! Searched for [{search_string}] in [{idx}] and did not find anything!"
            )
        )

def get_gefs(parameter, flist="default"):
    #print(f"Downloading {parameter} grids")
    func_start = datetime.now()
    if flist == "default":
        flist = fh_list

    for t in [i for i in flist if i <= 240]:
        for member in members_tqdm:
            members_tqdm.set_description(f"Downloading {parameter}: (FH{t} / {member})")
            # remote_file = f"ge{member}.t{model_run.strftime('%Hz')}.pgrb2s.0p25.f{int(t):03}"
            # file_name = prefix+remote_file
            # print(file_name)
            # uri = f"simplecache::s3://{bucket}/{file_name}"
            # file = fsspec.open_local(uri, s3={'anon': True}, filecache={'cache_storage':'tmpcache/'})
            local_file = f"tmp_{parameter}_{member}.grib2"
            level = "surface"
            download_subset(parameter, level, member, t, local_file)
            # s3.download_file(bucket, file_name, local_file)

            dataset_member = xr.open_dataset(
                os.path.join("tmpcache", local_file),
                engine="cfgrib",
                # backend_kwargs={"filter_by_keys": {'typeOfLevel': 'surface'}},
            )
            # dataset_member = dataset_member[sfc_variables] # only want a few variables
            dataset_member = dataset_member.expand_dims(
                {"step": 1}
            )  # need to add this as a dimension so we can append times
            dataset_member = dataset_member.expand_dims(
                {"number": 1}
            )  # need to add this as a dimension so we can append members
            if member == "c00":
                dataset = dataset_member
            else:
                dataset = xr.concat([dataset, dataset_member], dim="number")
            # dataset_sfc = dataset_sfc.sel(longitude=slice(-180, -30), latitude=slice(85, 5))
            # Now we want to store those x-array objects into a zarr, for fast, lazy access later
        if os.path.exists(os.path.join("data", f"gefs_{parameter}.zarr")):
             dataset.to_zarr(
                store=os.path.join("data", f"gefs_{parameter}.zarr"),
                mode="a",
                append_dim="step",
            )
        else:
            dataset.to_zarr(
                store=os.path.join("data", f"gefs_{parameter}.zarr"), mode="w"
            )
        try:
            tempfiles = glob.glob(os.path.join("tmpcache", f"tmp_{parameter}*"), recursive=True)
            for tempfile in tempfiles:
                os.remove(tempfile)
        except:
            pass

    func_end = datetime.now()
    func_total = func_end - func_start
    total_seconds = func_total.total_seconds()
    fminutes, fseconds = divmod(total_seconds, 60)
    print(f"{parameter} download complete in: {round(fminutes)}m {round(fseconds)}s")

latloncrs = ccrs.PlateCarree()
#proj = ccrs.epsg(3857)
if dom == "CONUS":
  proj = ccrs.AlbersEqualArea(central_longitude=-98.35,
                              central_latitude=39.5,
                              false_easting=0.0,
                              false_northing=0.0,
                              standard_parallels=(20.0, 50.0),
                              globe=None)
else:
  proj = ccrs.Mercator.GOOGLE

width = 7 # sets figure width value
height = 7 # sets figure height value

domain_dict = {
               "CONUS":{"west":-123.650,
                    "south":23.377,
                    "east":-71.488,
                    "north":50.924,
                    "zoom_adj": 0,
                    "legend":4},

               "ECONUS":{"west":-104.36,
                    "south":24.735,
                    "east":-66.453,
                    "north":49.755,
                    "zoom_adj": 0,
                    "legend":4},

               "WR":{"west":-126.917,
                    "south":30.586,
                    "east":-102.740,
                    "north":49.755,
                    "zoom_adj": 1,
                     "legend":4},

               "UT":{"west":-117.02,
                      "east":-106.92,
                      "north":42.13,
                      "south":36.80,
                      "zoom_adj": 1,
                     "legend":4},

               "NR":{"west":-117.5177,
                    "south":41.9071,
                    "east":-103.38071,
                    "north":49.3085,
                    "zoom_adj": 1,
                    "legend":4},

               "PNW":{"west":-125.4510,
                    "south":41.8754,
                    "east":-110.9318,
                    "north":49.5767,
                    "zoom_adj": 0,
                    "legend":4},

               "SWUS":{"west":-125.582,
                    "south":31.136,
                    "east":-108.689,
                    "north":42.859,
                    "zoom_adj": 0,
                    "legend":3},

               "AZ":{"west":-115.596,
                    "south":31.113,
                    "east":-107.887,
                    "north":37.446,
                    "zoom_adj": 0,
                    "legend":4},


               "CR":{"west":-111.534,
                    "south":35.118,
                    "east":-82.263,
                    "north":49.755,
                    "zoom_adj": 1,
                    "legend":4},

               "NP":{"west":-105.244,
                    "south":42.173,
                    "east":-89.426,
                    "north":49.474,
                    "zoom_adj": 1,
                    "legend":4},

               "GL":{"west":-97.606,
                    "south":38.735,
                    "east":-74.916,
                    "north":49.292,
                    "zoom_adj": 1,
                    "legend":3},

               "CUS":{"west":-111.553,
                    "south":34.794,
                    "east":-88.533,
                    "north":46.357,
                    "zoom_adj": 1,
                    "legend":3},

               "CO":{"west":-109.5,
                    "south":36.5,
                    "east":-101.5,
                    "north":41.5,
                    "zoom_adj": 1,
                    "legend":4},

               "SR":{"west":-109.758,
                    "south":23.313,
                    "east":-78.247,
                    "north":37.899,
                    "zoom_adj": 1,
                    "legend":3},

               "TXOK":{"west":-106.95,
                    "south":26.06,
                    "east":-86.76,
                    "north":37.76,
                    "zoom_adj": 1,
                    "legend":4},

               "SE":{"west":-92.974,
                    "south":24.578,
                    "east":-75.1311,
                    "north":37.390,
                    "zoom_adj": 1,
                    "legend":4},

               "ER":{"west":-85.629,
                    "south":31.723,
                    "east":-66.465,
                    "north":47.676,
                    "zoom_adj": 0,
                    "legend":4},

               "NE":{"west":-85.629,
                    "south":37.654,
                    "east":-66.00,
                    "north":47.825,
                    "zoom_adj": 0,
                    "legend":4},
}

west = domain_dict[dom]["west"]
south = domain_dict[dom]["south"]
east = domain_dict[dom]["east"]
north = domain_dict[dom]["north"]
#map_zoom_offset = domain_dict[dom]["zoom_adj"]
LLOC = domain_dict[dom]["legend"]

def blankmap():
    global maplayertext1, maplayertext2
    print(" > Initializing map")
    plt.figure(figsize=(width,height),frameon=True, facecolor=face_color)
    F = plt.gcf()  # Gets the current figure
    ax = plt.axes(projection=proj)

  ### Here is where you set up the domain.
  ### Want to add another? Just copy the last (elif) one and change the bounds (try to keep it square)
  ### Note the attributes are turned OFF on the cx.add_basemap layers IF you have mixed and matched provider sources. \
  ### This is because each attribution goes on top of the other, and thus are manually added so they remain legible.
    zoom = (cx.tile._calculate_zoom(west, south, east, north) - map_zoom_offset)

    ax.set_extent([west, east, south, north], crs=latloncrs)

    print(' > Adding fancy map tiles')
    maplayertext2 = ""
    maylayertext1 = ""
    if map_theme == "Light Shaded Relief":
      ax.add_feature(cfeature.LAND, edgecolor='none', facecolor='#FAFAF8', zorder=-2)
      cx.add_basemap(ax, source="https://basemap.nationalmap.gov/arcgis/rest/services/USGSShadedReliefOnly/MapServer/tile/{z}/{y}/{x}",
                      attribution=False, crs=proj, alpha =0.8, zorder=-1)
      maplayertext1 = "Map tiles: © USGS Earth Resources Observation & Science (EROS) Center: GMTED2010" #for USGS Hillshade
      ax.add_feature(cfeature.OCEAN, edgecolor='none', facecolor='#b3bbbd', zorder=2) # adds fill over the ocean
      ax.add_feature(cfeature.LAKES, edgecolor='none', facecolor='#b3bbbd', zorder=2) # adds fill over lakes
      cx.add_basemap(ax, source="http://services.arcgisonline.com/arcgis/rest/services/Reference/World_Boundaries_and_Places/MapServer/tile/{z}/{y}/{x}",
                      attribution=False, crs=proj, zoom=zoom, zorder=50)
      cx.add_basemap(ax, source='http://services.arcgisonline.com/arcgis/rest/services/Reference/World_Transportation/MapServer/tile/{z}/{y}/{x}',
                      crs=proj, zoom=zoom, zorder=49)
      maplayertext2 = "Esri, HERE, Garmin, OpenStreetMap contributors"

    elif map_theme == "Stamen Toner Light":
      cx.add_basemap(ax, source="https://tiles.stadiamaps.com/tiles/alidade_smooth/{z}/{x}/{y}{r}.png?api_key=c1f1f8dc-47a0-49e2-bd54-41ad85c19841", zoom=zoom, zorder=-1, attribution=False, crs=proj)
      #cx.add_basemap(ax, source=cx.providers.Stamen.TonerHybrid, zoom=zoom, zorder=49, attribution=False, crs=proj)
      maplayertext1 = str(cx.providers.Stadia.AlidadeSmooth.attribution)

    elif map_theme == "Positron":
      cx.add_basemap(ax, source=cx.providers.CartoDB.PositronNoLabels, zorder=-1, attribution=False, crs=proj)
      ax.add_feature(cfeature.NaturalEarthFeature('cultural', 'admin_1_states_provinces_lines', '50m', edgecolor='#e5e2e3', facecolor='none', linewidth=0.5, zorder=48, alpha=0.5)) # adds state borders
      ax.add_feature(cfeature.COASTLINE, edgecolor="#FAFAF8", facecolor=None, linewidth=0.25, alpha=0.5, zorder=48) # adds coastline (since that isn't included)
      ax.add_feature(cfeature.BORDERS, edgecolor="#FAFAF8", facecolor=None, linewidth=0.5, alpha=0.75, zorder=48) # adds country borders (since that isn't included)
      cx.add_basemap(ax, source=cx.providers.CartoDB.PositronOnlyLabels, zoom=zoom, zorder=49, attribution=False, crs=proj)
      ax.add_feature(cfeature.OCEAN, edgecolor='none', facecolor='#d5dadc', zorder=2) # adds fill over the ocean
      ax.add_feature(cfeature.LAKES, edgecolor='none', facecolor='#d5dadc', zorder=2) # adds fill over lakes
      maplayertext1 = "Map tiles: " + str(cx.providers.CartoDB.Positron.attribution)

    elif map_theme == "Dark Grey Matter":
      ax.add_feature(cfeature.LAND, edgecolor='none', facecolor='#414143', zorder=-2)
      ax.add_feature(cfeature.OCEAN, edgecolor='none', facecolor='#232227', zorder=2) # adds fill over the ocean
      ax.add_feature(cfeature.LAKES, edgecolor='none', facecolor='#232227', zorder=2) # adds fill over lakes
      #cx.add_basemap(ax, source=cx.providers.CartoDB.DarkMatterNoLabels, zorder=-1, attribution=False)
      ax.add_feature(cfeature.NaturalEarthFeature('cultural', 'admin_1_states_provinces_lines', '50m', edgecolor='#1c1c1c', facecolor='none', linewidth=0.5, zorder=48, alpha=0.5)) # adds state borders
      ax.add_feature(cfeature.COASTLINE, edgecolor='k', facecolor=None, linewidth=0.25, alpha=0.5, zorder=48) # adds coastline (since that isn't included)
      ax.add_feature(cfeature.BORDERS, edgecolor='k', facecolor=None, linewidth=0.5, alpha=0.75, zorder=48) # adds country borders (since that isn't included)
      cx.add_basemap(ax, source=cx.providers.CartoDB.DarkMatterOnlyLabels, zoom=zoom, zorder=49, attribution=False, crs=proj)
      maplayertext1 = "Map tiles: " + str(cx.providers.CartoDB.DarkMatter.attribution)

    elif map_theme == "Dark Matter":
      cx.add_basemap(ax, source=cx.providers.CartoDB.DarkMatterNoLabels, zorder=-1, attribution=False, crs=proj)
      ax.add_feature(cfeature.NaturalEarthFeature('cultural', 'admin_1_states_provinces_lines', '50m', edgecolor='#1c1c1c', facecolor='none', linewidth=0.5, zorder=48, alpha=0.5)) # adds state borders
      ax.add_feature(cfeature.COASTLINE, edgecolor='k', facecolor=None, linewidth=0.25, alpha=0.5, zorder=48) # adds coastline (since that isn't included)
      ax.add_feature(cfeature.BORDERS, edgecolor='k', facecolor=None, linewidth=0.5, alpha=0.75, zorder=48) # adds country borders (since that isn't included)
      cx.add_basemap(ax, source=cx.providers.CartoDB.DarkMatterOnlyLabels, zoom=zoom, zorder=49, attribution=False, crs=proj)
      maplayertext1 = "Map tiles: " + str(cx.providers.CartoDB.DarkMatter.attribution)

    elif map_theme == "ESRI Light Grey":
      cx.add_basemap(ax, source='https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}', zoom=zoom, zorder=-1, crs=proj, attribution=False)
      ax.add_feature(cfeature.NaturalEarthFeature('cultural', 'admin_1_states_provinces_lines', '50m', edgecolor='#e4e4e4', facecolor='none', linewidth=0.5, zorder=48)) # adds state borders
      ax.add_feature(cfeature.COASTLINE, edgecolor='#efefef', facecolor=None, linewidth=0.25, alpha=0.5, zorder=48) # adds coastline (since that isn't included)
      ax.add_feature(cfeature.BORDERS, edgecolor='#efefef', facecolor=None, linewidth=0.5, alpha=0.75, zorder=48) # adds country borders (since that isn't included)
      cx.add_basemap(ax, source='https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Reference/MapServer/tile/{z}/{y}/{x}', zoom=zoom, zorder=49, crs=proj, attribution=False)
      maplayertext1 = "Map tiles: Esri, HERE, Garmin, FAO, NOAA, USGS, © OpenStreetMap contributors, and the GIS User Community"

    elif map_theme == "ESRI Dark Grey":
      cx.add_basemap(ax, source='https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Dark_Gray_Base/MapServer/tile/{z}/{y}/{x}', zorder=-1, crs=proj, attribution=False)
      ax.add_feature(cfeature.NaturalEarthFeature('cultural', 'admin_1_states_provinces_lines', '50m', edgecolor='#363638', facecolor='none', linewidth=0.5, zorder=48, alpha=0.5)) # adds state borders
      ax.add_feature(cfeature.COASTLINE, edgecolor='#3f3f3f', facecolor=None, linewidth=0.25, alpha=0.5, zorder=48) # adds coastline (since that isn't included)
      ax.add_feature(cfeature.BORDERS, edgecolor='#3f3f3f', facecolor=None, linewidth=0.5, alpha=0.75, zorder=48) # adds country borders (since that isn't included)
      cx.add_basemap(ax, source='https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Dark_Gray_Reference/MapServer/tile/{z}/{y}/{x}', zoom=zoom, zorder=49, crs=proj, attribution=False)
      maplayertext1 = "Map tiles: Esri, HERE, Garmin, © OpenStreetMap contributors, and the GIS User Community"

    center_path = ShapelyFeature(shpreader.Reader("shp/center.shp").geometries(),
                                ccrs.PlateCarree(), facecolor='none', edgecolor=font_color, linestyle="dashed", linewidth=0.75, zorder=5)
    umbra_path = ShapelyFeature(shpreader.Reader("shp/upath_hi.shp").geometries(),
                                ccrs.PlateCarree(), facecolor='#0000001A', edgecolor=font_color, linewidth=0.5, zorder=5)
    ax.add_feature(center_path)
    ax.add_feature(umbra_path)



### Set up the plotting function (the thing that actually generates the finished figure)
def drawmap(DATA,TITLESTRING,UNITS,LEVS,valid_date):
    print(" > Finishing up map and adding legend")
    F = plt.gcf()  # Gets the current figure
    ax = plt.gca()  # Gets the current axes
    proxy = [mpatches.Patch(color = pc.get_facecolor()[0]) for pc in DATA.collections]
    LLABS = []
    for i in range(0, len(LEVS)-1):
      label = str(LEVS[i])+"-"+str(LEVS[i+1])+UNITS
      LLABS.append(label)

    proxy = proxy[::-1]
    LLABS = LLABS[::-1]

    LLABS[0] = "(High Chance Sunny)\n" + LLABS[0]
    LLABS[-1] = LLABS[-1] + "\n(Low Chance Sunny)"

    l = plt.legend(handles=proxy, labels=LLABS, fontsize='5',fancybox=True, loc=LLOC)
    for text in l.get_texts():
      text.set_color(font_color)
    lframe = l.get_frame()
    lframe.set_color(face_color)
    l.set_zorder(100)

    #artists, labels = DATA.legend_elements()

    init_title = model_run.strftime('%HZ %d-%b-%Y')
    gefs_text = r"$\bf{"+'GEFS'+"}$" +' (Global Ensemble Forecast System) · ' +r"$\bf{"+"Init"+"}$" +" "+ init_title
    valid_date_title = valid_date.strftime('%HZ %a\n %b %d %Y')
    title_text = r"$\bf{"+ TITLESTRING +"}$" +' (Less Than ' + str(sky_threshold) + r'% Cloud Cover)'

    plt.text(0.0,1.03, title_text, transform=ax.transAxes, ha='left', va='bottom', fontsize=9, color=font_color)
    plt.text(0.0,1.03, gefs_text, transform=ax.transAxes, ha='left', va='top', fontsize=6, color='grey')
    plt.text(1.0,1.03,valid_date_title, transform=ax.transAxes, ha='right', va='center', weight='bold', fontsize=7, color=font_color)

    if maplayertext1 != "":
      if maplayertext2 !="":
          plt.text(0.5, -0.009, '%s, %s // Graphic by Caleb Steele with Wx4Colab' % (maplayertext1,maplayertext2),
                  transform = ax.transAxes, ha='center', va ='top',fontsize=4,color=font_color, style='italic', zorder=99,
                  bbox=dict(facecolor=face_color,edgecolor='none', pad=1.8, alpha=0.65))
      else:
        plt.text(0.5, -0.009, '%s // Graphic by Caleb Steele with Wx4Colab' % (maplayertext1),
                  transform = ax.transAxes, ha='center', va ='top',fontsize=4,color=font_color, style='italic', zorder=99,
                  bbox=dict(facecolor=face_color,edgecolor='none', pad=1.8, alpha=0.65))

    file_id = 'GEFS_psky_%s_%sT%s' % (dom, model_run.strftime('%Y%m%d%H'), valid_date.strftime('%Y%m%d%H'))

    filename = '%s.png' % (file_id)
    print(f' > Saving plot as {filename}')
    plt.savefig(filename,bbox_inches='tight', facecolor=face_color) # Saves the figure with small margins

    plt.show()


def plot_tcdc_prob():
    print("Calculating probability and making plot...")
    blankmap()
    tcdc = gefs.tcc
    #print(f"St Louis: {gefs.sel(latitude=38.632, longitude=-90.208, method='nearest').tcc.values}")
    thresh_flag = xr.where(tcdc < sky_threshold, 1, 0)
    prob_clear = ((thresh_flag.sum(dim="number")) / len(thresh_flag.number.values)) * 100
    pclear_plot = plt.contourf(
                            gefslons,
                            gefslats,
                            prob_clear,
                            [0, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
                            cmap=plt.get_cmap("RdBu"),
                            #cmap=plt.get_cmap("RdYlGn"),
                            alpha=0.75,
                            transform=latloncrs,
                            transform_first=True,
                            antialiased=True
                  )

    valid_time = model_run + timedelta(hours=gefs.step.values.astype('timedelta64[h]') / np.timedelta64(1, 'h'))
    #title = f"Chance of Sky Cover < {sky_threshold}%"
    title = f"Chance\ of\ Clear\ Skies"
    units = r'%'
    levs = [0, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

    drawmap(pclear_plot, title, units, levs, valid_time)

if os.path.exists(os.path.join("data", "gefs_tcdc.zarr")):
  pass
else:
  get_gefs("tcdc")

gefs_ds = xr.open_zarr(os.path.join("data", "gefs_tcdc.zarr"))
new_lon = []
for coord in gefs_ds.longitude:
    if coord.values > 180:
        new_lon.append(coord.item(0) - 360.0)
    else:
        new_lon.append(coord.item(0))
gefs_ds = gefs_ds.assign_coords(longitude=new_lon)
gefs_ds = gefs_ds.sortby("longitude")
gefs_ds = gefs_ds.sortby("latitude")

matplotlib.rcParams['figure.dpi'] = 200
for fh in fh_tds:
  if fh > timedelta(hours=240):
    break
  gefs = gefs_ds.sel(step=fh, latitude=slice(south - 5, north + 5), longitude=slice(west - 5, east + 10))
  gefslons, gefslats = np.meshgrid(gefs.longitude.values, gefs.latitude.values)
  plot_tcdc_prob()
